## ODE version of the chemostat system



$$\left\{\begin{array}{rlll}
            x'(t) &= \big(M(s(t))-u(t)I_n+ɛT(s)\big)x(t), & t>0,\\
            s'(t) &= - \displaystyle\sum_{j=1}^n\mu_j(s(t))\frac{x_j(t)}{Y_j}+ u(t)(s_{in}-s(t)),& t>0 \\
            s(0)&=s_0, \\
            x(0)&=x_0.
        \end{array}
\right.$$

Where
$$ M(s) = \left[\begin{array}{cccc} \mu_1(s) & 0 & \cdots & 0 \\ 0 & \mu_2(s) & \cdots & 0 \\ \vdots & \ddots & \ddots & \vdots \\ 0 & \cdots & 0 &  \mu_n(s) \end{array}\right]$$
and
$$ T(s) = \left[\begin{array}{ccccc} -2\mu_1(s) & \mu_2(s) & 0 & \cdots & \mu_n(s)\\ \mu_1(s) & -2\mu_2(s) & \mu_3(s) & \cdots & 0\\ \vdots & \ddots & \ddots & \ddots & \vdots\\ 0 & \cdots & \mu_{n-2}(s) &  -2\mu_{n-1}(s) & \mu_n(s)\\ \mu_1(s)& \cdots & 0& \mu_{n-1}(s)& -2\mu_n(s) \end{array}\right] \qquad\text{or}\qquad T \equiv \left[\begin{array}{ccccc} -1 & 1 & 0 & \cdots & 0\\ 1 & -2 & 1 & \cdots & 0\\ \vdots & \ddots & \ddots & \ddots & \vdots\\ 0 & \cdots & 1 &  -2 & 1\\ 0& \cdots & 0& 1& -1 \end{array}\right]$$

* Monod Case: $\mu_i(s)= \displaystyle\frac{a_i s}{b_i+s}$

For *Monod kinetics*, consider $λ$ as the greatest eigenvalue function. We have the following for each pair $(ɛ,u)$:

- $B(s,ɛ,u)= M(s)-uI_n+ɛT$
- $s^{ɛ,u}$ is the unique solution of $\lambda(B(s,ɛ,u))=0$
- $x^{ɛ,u}$ is the Perron vector of $B(s^{ɛ,u},ɛ,u)$ such that $\sum x^{ɛ,u}_j+s^{ɛ,u}=1$
- $d_{ɛ,u}=(M'(s^{ɛ,u})+ɛT'(s))x^{ɛ,u}$
- $A_{ɛ,u}=B(s^{ɛ,u},e,u)$
- $\mu^{ɛ,u}=(-\frac{\mu_1(s^{ɛ,u})}{Y_1},\ldots,-\frac{\mu_n(s^{ɛ,u})}{Y_n})$
- $u_{c}(ɛ)=λ(M(s_{in})+ɛ T)$


Then, the jacobian of the system is given by
$$ J_{ɛ,u}= \left[\begin{array}{cc}A_{ɛ,u}&d_{ɛ,u}\\\mu^{ɛ,u}&-u \end{array}\right]$$


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import brentq
from scipy.linalg import eigvals, eig

In [ ]:
#Model type
yields = "si" #"no"
kinetics = "monod"
# mutation = 1 # lineal coupling
mutation = 1 # cultivated biomass


In [ ]:
#Constant values
n = 5
s_in = 1
a = np.array([0.83577988, 0.46238014, 0.33884829, 0.48376133, 0.75915236])
b = np.array([0.27706137, 0.90180048, 0.10842917, 0.08861974, 0.36456998])

o = 0.1

Y = np.array([1.0,1.0,1.0,1.0,1.0]) if(yields=="no") else np.array([1.0,1.5,2.0,2.5,3.0]) #[0.1,0.1,0.1,0.1,0.1]

T = np.tril(np.triu(np.diag([-2]+[-3 for i in range(n-2)]+[-2])+1,-1),1)
T2 = np.diag([-2 for i in range(n)])+np.diag([1 for i in range(0,n-1)],-1)+np.diag([1 for i in range(1,n)],1)
T2[0][n-1] = 1
T2[n-1][0] = 1

In [ ]:
def mu(s,i):
    return a[i]*s/(b[i]+s)

### Plotting of the kinetics


In [ ]:
x = np.linspace(0,1,100)
y = [[mu(s,i) for s in x] for i in range(n)]

fig = go.Figure()

for i in range(n):
  fig.add_trace(go.Scatter(x=x,y=y[i],name=r"$\mu_{k}(s)$".format(k=i+1)))
  fig.add_trace(go.Scatter(x=[b**0.5],y=[mu(b**0.5,i)],showlegend=False, mode="markers"))

fig.update_layout(xaxis_title='s',yaxis_title='', 
                  width=600, height=300,
                  margin=dict(l=0,r=0,b=0,t=10))

fig.show()

In [ ]:
def B(s,e,u):
    D = np.diag(np.array([mu(s,i) for i in range(n)]))
    match mutation:
        case 1:
            return D-u*np.identity(n)+e*T
        case 2:
            return D-u*np.identity(n)+(e/2)*np.dot(T2,D)

def s_B(e,u):
    f = lambda s: max(np.real(eigvals(B(s,e,u)))) #Maximum eigenvalue of the symmetric matrix B
    return brentq(f, 0, s_in) #lambda(B(s))=0.

def s_states(e,u):
    sB = s_B(e,u)
    Beu = B(sB,e,u)
    eigval, eigvect = eig(Beu)
    a_eu = np.real(eigvect[:, np.argmax(np.real(eigval))])
    alpha_eu = (u*(s_in-sB))/(sum([mu(sB,i)*a_eu[i]/Y[i] for i in range(n)]))
    x_eu = alpha_eu*a_eu
    return x_eu, sB

def f(t,y,e,u):
    x, s = y[0:n], y[n]
    B_seu = B(s,e,u)
    dx = np.dot(B_seu,x)
    ds = -sum([mu(s,i)*x[i]/Y[i] for i in range(n)])+u*(s_in-s)
    return np.concatenate([dx,[ds]])



In [ ]:
def dmu(s,i):
    return (a[i]*b[i])/((b[i]+s)**2)

def A(e,u):
    x, s = s_states(e,u)
    b_eu = B(s,e,u)
    d_eu = np.array([dmu(s,i)*x[i] for i in range(n)])
    if mutation==2:
        dT = np.dot(T2,np.diag(np.array([dmu(s,i) for i in range(n)])))
        d_eu += (e/2)*np.dot(dT,x)
    mu_eu = np.concatenate(([-mu(s,i)/Y[i] for i in range(n)],[-u-sum([dmu(s,i)*x[i]/Y[i] for i in range(n)])]))
    
    return np.r_[np.c_[b_eu,d_eu],[mu_eu]], x

###  $u_c$ computation

In [ ]:
T_0 = 0
T_f = 200

mu_S_in = np.array([mu(s_in,i) for i in range(n)])
mu_max = max(mu_S_in)

D_sin = np.diag(np.array([mu(s_in,i) for i in range(n)]))
V_sin = 0.5*np.dot(T2,D_sin) if mutation==2 else T

u_c = lambda eps : max(np.real(eigvals(D_sin+eps*V_sin)))

eigval, eigvect_left, eigvect_right = eig(-V_sin, left=True)
i_valmax = np.argmax(np.real(eigvals(V_sin)))
vT_left, vT_right = eigvect_left[:,i_valmax], eigvect_right[:,i_valmax]
vT_rl = np.sqrt(np.dot(vT_left,vT_right))
vT_left, vT_right = vT_left/vT_rl, vT_right/vT_rl
mu_mean = np.dot(vT_left,np.dot(D_sin,vT_right))


In [ ]:
e_list = np.linspace(0.0,40.0,100)
u_c_list = np.array([u_c(e) for e in e_list])
fig = go.Figure()
fig.add_trace(go.Scatter(x=e_list,y=u_c_list, name=r"$u_c(\varepsilon)$"))
fig.add_trace(go.Scatter(x=[e_list[-1]],y=[mu_mean], name=r"$\hat{\mu}$", mode="markers"))
fig.add_trace(go.Scatter(x=[e_list[0]],y=[mu_max], name=r"$\mu_{\max}$", mode="markers"))
fig.update_layout(title="",xaxis_title=r"$\varepsilon$",yaxis_title=r"$u$", 
                  width=400, height=200,margin=dict(l=0,r=0,b=0,t=10))
fig.show()

#### One simulation

In [ ]:
u = 0.2
e = 0.1

x_0 = [0.15 for i in range(n)]

s_0 = 0.25

y0 = np.concatenate((x_0,[s_0]))
t = np.linspace(T_0,T_f,150)
sol = solve_ivp(f,[T_0,T_f],y0,t_eval=t,args=(e,u))

X = np.array([sol.y[i] for i in range(n)])
S = sol.y[n]

x_star, s_star = s_states(e,u)

y_star = np.concatenate((x_star, [s_star]))


In [ ]:
x_labels = ["x₁(t)","x₂(t)","x₃(t)","x₄(t)","x₅(t)"] 
fig = go.Figure()
for i in range(n):
  fig.add_trace(go.Scatter(x=t,y=X[i],name=f"$x_{i+1}(t)$"))
fig.add_trace(go.Scatter(x=t,y=S,name=r"$s(t)$"))
if kinetics=="monod":
  fig.add_trace(go.Scatter(x=[t[-1] for i in range(n)],y=x_star,
                           name=r"$x^{\varepsilon,u}$", 
                           mode="markers",marker_line_width=2,
                           marker_color="red",marker_symbol="diamond",
                           marker_line_color="darkred",marker_size=5))
  fig.add_trace(go.Scatter(x=[t[-1]],y=[s_star],
                           name=r"$s^{\varepsilon,u}$", 
                           mode="markers", line_color="black",
                           marker_symbol="square",marker_line_width=2,
                           marker_color="red",marker_line_color="darkred",marker_size=5))
fig.update_layout(xaxis_title='t',yaxis_title='', 
                  width=600, height=300,
                  margin=dict(l=0,r=0,b=0,t=10))

fig.show()



In [ ]:
Biomass = np.round(sum(X)+S,5)
fig = go.Figure()
fig.add_trace(go.Scatter(x=t,y=Biomass))
fig.update_layout(title="Total Biomass",
                  xaxis_title='t',
                  yaxis_title='', 
                  width=600, height=300,
                  margin=dict(l=0,r=0,b=0,t=50))
fig.show()

### Distance of trayectories and $x_{u,e}$ or $x_{wo}$

$$d(x_0)(T) = \| x(x_0,T)-x_{\varepsilon,u} \| $$



In [ ]:
n_hp = 50
e_list = np.linspace(0.0,5.0,n_hp)
u_list = np.linspace(0.0,0.8,n_hp)
hp_distances = np.zeros((n_hp,n_hp))
E_0 = np.zeros(n+1)
E_0[n] = s_in

In [ ]:
u_c_list = np.array([u_c(e) for e in e_list])
T_0 = 0
T_f = 200
t = np.linspace(T_0,T_f,3)

In [ ]:
i = 0
j = 0
for u in u_list:
    for e  in e_list:
        sol = solve_ivp(f,[T_0,T_f],y0,t_eval=t,args=(e,u))
        E_T = sol.y[:,-1]
        hp_distances[i][j] = np.linalg.norm(E_0-E_T)
        j += 1
    i += 1
    j = 0

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True,"r":0.05}]],shared_xaxes=True)

fig.add_trace(go.Heatmap(
        z=hp_distances,
        x=e_list,
        y=u_list,
        colorscale='Viridis',
        colorbar={"titleside":'bottom',  
                  "titlefont":dict(size=14, family='Times New Roman'),
                  "x":.94,"y":0.53}))

fig.add_trace(go.Scatter(
    mode='lines', 
    x=e_list, 
    y=u_c_list,
    line=dict(color='red',width=2),
    legend="legend2",
    showlegend=True, name=r"$u_c(\varepsilon)$"), secondary_y=False,)
              
fig.update_layout(xaxis_title=r'$\varepsilon$',
                  yaxis_title=r'$u$', 
                  width=600, height=300,
                  margin=dict(l=0,r=0,b=0,t=0), 
                  legend2={
                    "y": -0.1,
                    "x": 0.95
                    }
                  )

fig.show()

#### Hurwitz comprobation

In [ ]:
n_hw = 150
e_vector = np.linspace(0.0,5.0,n_hw)
u_vector = np.linspace(0.001,0.7,n_hw)

hurwitz = np.zeros((n_hw,n_hw))
X = np.zeros((n_hw,n_hw))

i = 0
for ei in e_vector:
    j = 0
    for uj in u_vector:
        try:
          A_val, _ = A(ei,uj)
          eigval = max(np.real(eigvals(A_val)))
        except:
          eigval = None
        hurwitz[i][j] =  eigval
        j += 1
    i += 1

In [ ]:
u_c_vector = np.array([u_c(e) for e in e_vector])

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True,"r":0.05}]],shared_xaxes=True)

fig.add_trace(go.Heatmap(
        z=np.transpose(hurwitz),
        x=e_vector,
        y=u_vector, colorscale='Plasma',
        colorbar={"titleside":'bottom',  
                  "titlefont":dict(size=14, family='Times New Roman'),
                  "x":.94,"y":0.53}))

fig.add_trace(go.Scatter(
    mode='lines', 
    x=e_vector, 
    y=u_c_vector,
    line=dict(color='red',width=2),
    legend="legend2",
    showlegend=True, name=r"$u_c(\varepsilon)$"), secondary_y=False,)
              
fig.update_layout(xaxis_title=r'$\varepsilon$',
                  yaxis_title=r'$u$', 
                  width=605, height=300,
                  margin=dict(l=0,r=0,b=0,t=0), 
                  legend2={
                    "y": -0.1,
                    "x": 0.86
                    }
                  )
fig.show()

### Simulations varing the initial conditions

In [ ]:
e = .1
u = .4
x_star, s_star = s_states(e,u) if(kinetics=="monod" and mutation != 2) else (0,0)
y_star = np.concatenate((x_star, [s_star]))

Len_T = 400
T_f = 100
T_0 = 0
t = np.linspace(T_0,T_f,Len_T)

xi_list = list(np.linspace(0.0,2.0,21)) #when e=0, x1(0)>0.001
N_IC = 100
list_IC = [random.sample(xi_list, n) for j in range(N_IC)]

Graph = {i : np.zeros((N_IC,Len_T)) for i in range(n+1)}

for j in range(N_IC):
    x0 = list_IC[j]
    s0 = 1-sum(x0)
    if s0<=0:
        s0 = 0.9
    y0 = np.concatenate((x0,[s0]))
    sol = solve_ivp(f,[T_0,T_f],y0,t_eval=t,args=(e,u))
    for i in range(n):
        Graph[i][j] = sol.y[i]
    Graph[n][j] = sol.y[n] #s values

In [ ]:
row_column = {0:(1,2),1:(1,3),2:(2,1),3:(2,2),4:(2,3),5:(1,1)}
fig = make_subplots(rows=2, cols=3)

for i in range(n+1):
    graph_i = Graph[i]
    r,c = row_column[i]
    for j in range(N_IC):
        y = graph_i[j]
        fig.add_trace(go.Scatter(x=t,y=y,showlegend=False,mode="lines", line_width=0.5), row=r,col=c)
    if kinetics=="monod" and mutation!=2:
        text=r"$\hspace{{-0.2cm}}x^{{\varepsilon,u}}_{i}$".format(i=i+1) if(i<n) else r"$s^{{\varepsilon,u}}$"
        fig.add_trace(go.Scatter(x=[T_f],y=[y_star[i]], text=text,mode="markers+text",marker_line_width=0, textposition="top left",
                        marker_color="black",marker_size=6,showlegend=False), row=r,col=c)
    fig.update_xaxes(title_text=r"$t$", row=r, col=c, range=[-1, T_f+5],title_standoff = 5)
    title_y = r"$x_{i}$".format(i=i+1) if(i<n) else r"$s$"
    s_off = 0 if(i<n) else 5
    fig.update_yaxes(title_text=title_y, row=r, col=c, title_standoff = s_off)
fig.update_layout(width=900, height=450,margin=dict(l=20,r=20,b=20,t=20))

fig.show()

In [ ]:
alpha = 0.05
x1_list = list(np.linspace(0.0,alpha,21))#when e=0, x1(0)>0.001

xi_list = list(np.linspace(0.0,1.0,21))

N_IC = 50
list_IC1 = np.zeros((N_IC,n))
list_IC2 = np.zeros((N_IC,n))

m = 0
while(m<N_IC):
    x0 = random.sample(xi_list, n)
    sum_Y_x0 = sum([x0[i]/Y[i] for i in range(n)])
    if(sum_Y_x0<=s_in and x0[0]>alpha):
        list_IC1[m] = x0
        m += 1
m = 0
while(m<N_IC):
    x0 = np.concatenate((random.sample(x1_list, 1),random.sample(xi_list, n-1)))
    sum_Y_x0 = sum([x0[i]/Y[i] for i in range(n)])
    if sum_Y_x0<=s_in:
        list_IC2[m] = x0
        m += 1

In [ ]:
Len_T = 400
T_f = 150
T_0 = 0
t = np.linspace(T_0,T_f,Len_T)

Graph1 = {i : np.zeros((N_IC,Len_T)) for i in range(n+1)}
Graph2 = {i : np.zeros((N_IC,Len_T)) for i in range(n+1)}

for j in range(N_IC):
    x0 = list_IC1[j]
    sum_Y_x0 = sum([x0[i]/Y[i] for i in range(n)])
    x0_list = list(np.linspace(0.0,s_in-sum_Y_x0,21))
    s0 = random.sample(x0_list, 1)[0]
    if s0<0:
        s0 = 0.9
        print("s0 negativo")
    y0 = np.concatenate((x0,[s0]))
    sol = solve_ivp(f,[T_0,T_f],y0,t_eval=t,args=(e,u))
    for i in range(n):
        Graph1[i][j] = sol.y[i]
    Graph1[n][j] = sol.y[n] #s values

for j in range(N_IC):
    x0 = list_IC2[j]
    sum_Y_x0 = sum([x0[i]/Y[i] for i in range(n)])
    x0_list = list(np.linspace(0.0,s_in-sum_Y_x0,21))
    s0 = random.sample(x0_list, 1)[0]
    # s0 = 1-sum(x0)
    if s0<0:
        s0 = 0.9
        print("s0 negativo")
    y0 = np.concatenate((x0,[s0]))
    sol = solve_ivp(f,[T_0,T_f],y0,t_eval=t,args=(e,u))
    for i in range(n):
        Graph2[i][j] = sol.y[i]
    Graph2[n][j] = sol.y[n] #s values

In [ ]:
if(mutation != 3):
    row_column = {0:(1,2),1:(1,3),2:(2,1),3:(2,2),4:(2,3),5:(1,1)}
    fig = make_subplots(
        rows=2, cols=3,
        )
else:
    row_column = {0:(1,3),1:(2,1),2:(2,2),3:(2,3),4:(1,1)}
    fig = make_subplots(
        rows=2, cols=3,
        specs=[[{"colspan": 2}, None, {}],
               [{},{},{}]],
        subplot_titles=(r"$s$",r"$x_1$",r"$x_2$",r"$x_3$",r"$x_4$"))


for i in range(n+1):
    r,c = row_column[i]

    graph_i = Graph1[i]
    color = "magenta"
    for j in range(N_IC):
        y = graph_i[j]
        fig.add_trace(go.Scatter(x=t,y=y,showlegend=False,mode="lines", line_color=color, line_width=0.5), row=r,col=c)
    
    graph_i = Graph2[i]
    color = "orange"
    for j in range(N_IC):
        y = graph_i[j]
        fig.add_trace(go.Scatter(x=t,y=y,showlegend=False,mode="lines", line_color=color, line_width=0.5), row=r,col=c)
    
    if kinetics=="monod" and mutation!=2:
        txp = "top left" if(i != 0) else "bottom left"
        text=r"$x^{{\varepsilon,u}}_{i}$".format(i=i+1) if(i<n) else r"$s^{{\varepsilon,u}}$"
        fig.add_trace(go.Scatter(x=[T_f],y=[y_star[i]], text=text,mode="markers+text",marker_line_width=0, textposition=txp,
                        marker_color="black",marker_size=6,showlegend=False), row=r,col=c)
    fig.update_xaxes(title_text=r"$t$", row=r, col=c, range=[-1, T_f+5],title_standoff = 5)
    title_y = r"$x_{i}$".format(i=i+1) if(i<n) else r"$s$"
    s_off = 0 if(i<n) else 5
    fig.update_yaxes(title_text=title_y, row=r, col=c, title_standoff = s_off)
fig.update_layout(width=900, height=450,margin=dict(l=20,r=20,b=20,t=20))
fig.show()